In [ ]:
https://www.kaggle.com/code/iambideniz/hybrid-recommender-system#Let's-Start!

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix
from sklearn.metrics import davies_bouldin_score

In [38]:
df = pd.read_csv ('/content/drive/MyDrive/Recommendation/dfn3share.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head()

,user,service,count,subcat,gender
0,3646,92,127,15,1
1,3646,94,18,15,1
2,3646,93,19,15,1
3,3646,95,2,17,1
4,3646,114,1,15,1


In [39]:
df['Ratings'] = pd.cut(df['count'], bins=5, labels=[1, 2, 3,4,5])
df.head()

,user,service,count,subcat,gender,Ratings
0,3646,92,127,15,1,5
1,3646,94,18,15,1,1
2,3646,93,19,15,1,1
3,3646,95,2,17,1,1
4,3646,114,1,15,1,1


In [5]:
df["service"].nunique()

301

In [7]:
df["service"].value_counts().head(10)

92     108566
20      39640
139     27932
0       26916
42      24071
94      21523
98      19056
46      17031
155     16864
4       16157
Name: service, dtype: int64

In [40]:
min_ratings=1
df_top_songs = df[df['count']>=min_ratings]

df_ratings_with_top_songs = df[df['service'].isin(list(df_top_songs.index))]

In [41]:
min_songs_rated = 1

df_users = df_ratings_with_top_songs[['service','user']].groupby(['user']).agg(['count']).sort_values(('service','count'),ascending=False)

df_top_rating_users = df_users[df_users[('service', 'count')]>=min_songs_rated]

top_rating_users = list(df_top_rating_users.index)

df_final=df_ratings_with_top_songs[df_ratings_with_top_songs['user'].isin(top_rating_users)]

In [42]:
df_user_item_matrix = df_final.pivot(index='user',columns='service',values='count')

In [43]:
df_user_item_matrix.head()

service,0,1,2,3,4,5,6,7,8,9,...,555,560,562,568,580,581,583,585,586,588
user,,,,,,,,,,,,,,,,,,,,,
0,3.0,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9.0,1.0,NaN,1.0,2.0,NaN,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
service_name = df_user_item_matrix[92]
df_user_item_matrix.corrwith(service_name).sort_values(ascending=False).head(10)

/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2821: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2680: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


service
92     1.000000
562    0.714435
104    0.330721
127    0.324282
195    0.274300
349    0.258221
277    0.256498
96     0.214599
512    0.214330
553    0.198219
dtype: float64

In [45]:
random_user = 3646

In [46]:
# Let's reduce the #dataset to user 3646:
random_user_df = df_user_item_matrix[df_user_item_matrix.index == random_user]
random_user_df

service,0,1,2,3,4,5,6,7,8,9,...,555,560,562,568,580,581,583,585,586,588
user,,,,,,,,,,,,,,,,,,,,,
3646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# Let's choose non-NaN. service used by all 3646:
service_used = random_user_df.columns[random_user_df.notna().any()].tolist()
service_used

[92, 93, 94, 95, 114]

In [48]:
#let's verify:
df_user_item_matrix.loc[df_user_item_matrix.index == random_user, df_user_item_matrix.columns == 92]
# gave this movie a 3.0 rating.


service,92
user,
3646,127.0


In [51]:
# we have reduced the dataset based on movies watched by user 28491:
service_used_df = df_user_item_matrix[service_used]
service_used_df.head()
service_used_df.shape

(426026, 5)

In [55]:
# information on how many services each user used in total:
user_service_count = service_used_df.T.notnull().sum()

user_service_count = user_service_count.reset_index()
user_service_count.columns = ["user","service_count"]
user_service_count.head().sort_values(by='service_count',ascending=False)

,user,service_count
1,1,4
0,0,3
2,2,1
4,6,1
3,5,0


In [57]:


# 3 user watched 22 movies:
user_service_count[user_service_count["service_count"] == 2].count()



user             21471
service_count    21471
dtype: int64

In [64]:
# 60% of movies watched by 3646:
perc = len(service_used) * 60 / 100
perc

3.0

In [65]:
# People who have use more than 60% service together with 3646 users:
users_same_service = user_service_count[user_service_count["service_count"] > perc]["user"]
users_same_service.count()


104

In [66]:
# Let's combine the data of user #3646 and similar users:
final_df = pd.concat([service_used_df[service_used_df.index.isin(users_same_service)],
                      random_user_df[service_used]])

final_df.shape
final_df.T.corr()

user,1,521,1107,2323,3646,9560,9766,10359,10657,11224,...,763714,816756,854259,881065,896257,938083,1171161,1223835,1231672,3646
user,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.970269,NaN,0.993735,0.990460,-0.430637,1.000000,0.985569,0.916482,1.000000,...,0.992895,-0.593660,-0.382994,-0.098039,1.000000,NaN,0.193072,0.533526,0.993735,0.990460
521,0.970269,1.000000,NaN,0.997949,0.969650,-0.924473,0.997609,0.920150,0.832240,0.939302,...,0.991241,NaN,0.997949,-0.515073,0.997609,NaN,0.018229,-0.059358,0.172720,0.969650
1107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2323,0.993735,0.997949,NaN,1.000000,0.990793,-0.500000,1.000000,0.958423,0.870388,0.960769,...,1.000000,-0.333333,-0.088045,-0.397360,1.000000,NaN,0.244851,-0.183368,0.000000,0.990793
3646,0.990460,0.969650,NaN,0.990793,1.000000,-0.522893,0.990606,0.970806,0.892633,0.931603,...,0.989723,-0.260735,-0.013774,-0.149795,0.990606,NaN,0.302182,-0.099307,0.115086,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1171161,0.193072,0.018229,NaN,0.244851,0.302182,0.821995,1.000000,0.511336,0.690494,0.960769,...,0.082199,-0.538672,-0.504456,0.881895,1.000000,NaN,1.000000,-0.715373,-0.996616,0.302182
1223835,0.533526,-0.059358,NaN,-0.183368,-0.099307,-0.842777,-0.157135,-0.158912,-0.500085,0.118896,...,0.572637,-0.183368,-0.242170,-0.411103,-0.157135,NaN,-0.715373,1.000000,0.972868,-0.099307


In [67]:
#corr for all users:
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df = corr_df.reset_index()
corr_df.head()

,user_id_1,user_id_2,corr
0,611287,816756,-1.00000
1,314652,208807,-1.00000
2,754787,682467,-1.00000
3,289614,9560,-1.00000
4,398707,881065,-0.99991


In [70]:
# Users with a correlation of %65 or more with 3646 users:
top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.65)][
    ["user_id_2", "corr"]].reset_index(drop=True)

top_users = top_users.sort_values(by='corr', ascending=False)
top_users.rename(columns={"user_id_2": "user"}, inplace=True)
top_users

,user,corr
28,545102,0.991679
27,430643,0.991679
26,744586,0.990793
25,138450,0.990793
24,314652,0.990606


In [71]:
top_users_ratings = top_users.merge(df[["user", "service", "count"]], how='inner')

top_users_ratings = top_users_ratings[top_users_ratings["user"] != random_user]
top_users_ratings.head()

,user,corr,service,count
0,545102,0.991679,92,12
1,545102,0.991679,20,3
2,545102,0.991679,94,1
3,545102,0.991679,151,1
4,545102,0.991679,42,1


In [73]:
df.loc[df['user']==545102].head()

,user,service,count,subcat,gender,Ratings
5280,545102,92,12,15,1,1
5281,545102,20,3,80,1,1
5282,545102,94,1,15,1,1
5283,545102,151,1,16,1,1
5284,545102,42,1,4,1,1


In [82]:
# Calculate the Weighted Average Recommendation Score and keep the first 5 movies.

#Let's do a single score with the most similar by corr * rating:
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['count']
top_users_ratings.groupby('service').agg({"weighted_rating": "mean"})

recommendation_df = top_users_ratings.groupby('service').agg({"weighted_rating": "mean"})
recommendation_df = recommendation_df.reset_index()
recommendation_df.head().sort_values("weighted_rating", ascending=False)


,service,weighted_rating
0,0,2.747400
4,4,2.613019
1,1,2.285826
2,2,1.453962
3,3,0.990460


In [88]:
recommendation_df[recommendation_df["weighted_rating"] > 3.7]


,service,weighted_rating
41,52,4.108253
58,75,4.158000
69,92,9.063812
127,171,4.938826
131,175,4.945136
145,333,3.774575
147,337,4.653444
156,393,5.584133
178,508,6.514822


In [92]:
service_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 3.7].sort_values("weighted_rating", ascending=False)
service_to_be_recommend

,service,weighted_rating
69,92,9.063812
178,508,6.514822
156,393,5.584133
131,175,4.945136
127,171,4.938826
147,337,4.653444
58,75,4.158000
41,52,4.108253
145,333,3.774575


In [91]:
service_to_be_recommend['service'].to_list()

[92, 508, 393, 175, 171, 337, 75, 52, 333]

In [15]:
# Let's get to the movies with less than 1000 reviews:
comment_counts = pd.DataFrame(df["service"].value_counts())
rare_movies = comment_counts[comment_counts["service"] <= 5].index

# Let's get access to movies with over 1000 reviews:
common_movies = df[~df["service"].isin(rare_movies)]
common_movies.shape

AttributeError: ignored

In [10]:
common_movies["service"].nunique()

275

In [11]:


# Let's create the User Movie Df:
user_movie_df = common_movies.pivot_table(index=["user"], columns=["service"], values="Ratings")

# There are 3159 movies that 138493 users have voted for. 
user_movie_df.shape



<ipython-input-11-8204eb87607b>:2: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  user_movie_df = common_movies.pivot_table(index=["user"], columns=["service"], values="Ratings")


(426023, 0)

In [12]:
user_movie_df.head(10)

service
user
0
1
2
5
6
7
8
9
15
